ThinkDSP
========
by Allen Downey (think-dsp.com)

This notebook contains examples and demos for a SciPy 2015 talk.

In [ ]:
from __future__ import print_function, division

import thinkdsp
import thinkplot

import numpy

%matplotlib inline

A Signal represents a function that can be evaluated at an point in time.

In [ ]:
cos_sig = thinkdsp.CosSignal(freq=440)

A cosine signal at 440 Hz has a period of 2.3 ms.

In [ ]:
cos_sig.plot()
thinkplot.config(xlabel='time (s)', legend=False)

`make_wave` samples the signal at equally-space time steps.

In [ ]:
wave = cos_sig.make_wave(duration=0.5, framerate=11025)

`make_audio` creates a widget that plays the Wave.

In [ ]:
wave.apodize()
wave.make_audio()

`make_spectrum` returns a Spectrum object.

In [ ]:
spectrum = wave.make_spectrum()

A cosine wave contains only one frequency component (no harmonics).

In [ ]:
spectrum.plot()
thinkplot.config(xlabel='frequency (Hz)', legend=False)

A SawTooth signal has a more complex harmonic structure.

In [ ]:
saw_sig = thinkdsp.SawtoothSignal(freq=440)
saw_sig.plot()

Here's what it sounds like:

In [ ]:
saw_wave = saw_sig.make_wave(duration=0.5)
saw_wave.make_audio()

And here's what the spectrum looks like:

In [ ]:
saw_wave.make_spectrum().plot()

Here's a short violin performance from jcveliz on freesound.org:

In [ ]:
violin = thinkdsp.read_wave('92002__jcveliz__violin-origional.wav')
violin.make_audio()

The spectrogram shows the spectrum over time:

In [ ]:
spectrogram = violin.make_spectrogram(seg_length=1024)
spectrogram.plot(high=5000)

We can select a segment where the pitch is constant:

In [ ]:
start = 1.2
duration = 0.6
segment = violin.segment(start, duration)

And compute the spectrum of the segment:

In [ ]:
spectrum = segment.make_spectrum()
spectrum.plot()

The dominant and fundamental peak is at 438.3 Hz, which is a slightly flat A4 (about 7 cents). 

In [ ]:
spectrum.peaks()[:5]

As an aside, you can use the spectrogram to help extract the Parson's code and then identify the song.

Parson's code: DUUDDUURDR

Send it off to http://www.musipedia.org

A chirp is a signal whose frequency varies continuously over time (like a trombone).

In [ ]:
import math
PI2 = 2 * math.pi

class SawtoothChirp(thinkdsp.Chirp):
    """Represents a sawtooth signal with varying frequency."""

    def _evaluate(self, ts, freqs):
        """Helper function that evaluates the signal.

        ts: float array of times
        freqs: float array of frequencies during each interval
        """
        dts = numpy.diff(ts)
        dps = PI2 * freqs * dts
        phases = numpy.cumsum(dps)
        phases = numpy.insert(phases, 0, 0)
        cycles = phases / PI2
        frac, _ = numpy.modf(cycles)
        ys = thinkdsp.normalize(thinkdsp.unbias(frac), self.amp)
        return ys

Here's what it looks like:

In [ ]:
signal = SawtoothChirp(start=220, end=880)
wave = signal.make_wave(duration=2, framerate=10000)
segment = wave.segment(duration=0.06)
segment.plot()

Here's the spectrogram.

In [ ]:
spectrogram = wave.make_spectrogram(1024)
spectrogram.plot()
thinkplot.config(xlabel='time (s)', 
                 ylabel='frequency (Hz)', 
                 legend=False)

What do you think it sounds like?

In [ ]:
wave.apodize()
wave.make_audio()

Up next is one of the coolest examples in Think DSP.  It uses LTI system theory to characterize the acoustics of a recording space and simulate the effect this space would have on the sound of a violin performance.

I'll start with a recording of a gunshot:

In [ ]:
response = thinkdsp.read_wave('180960__kleeb__gunshot.wav')

start = 0.12
response = response.segment(start=start)
response.shift(-start)

response.normalize()
response.plot()
thinkplot.config(xlabel='time (s)', 
                 ylabel='amplitude', 
                 ylim=[-1.05, 1.05], 
                 legend=False)

If you play this recording, you can hear the initial shot and several seconds of echos.

In [ ]:
response.make_audio()

This wave records the "impulse response" of the room where the gun was fired.

Now let's load a recording of a violin performance:

In [ ]:
wave = thinkdsp.read_wave('92002__jcveliz__violin-origional.wav')
start = 0.11
wave = wave.segment(start=start)
wave.shift(-start)

wave.truncate(len(response))
wave.normalize()
wave.plot()
thinkplot.config(xlabel='time (s)', 
                 ylabel='amplitude', 
                 ylim=[-1.05, 1.05],
                 legend=False)

And listen to it:

In [ ]:
wave.make_audio()

Now we can figure out what the violin would sound like if it was played in the room where the gun was fired.  All we have to do is convolve the two waves:

In [ ]:
output = wave.convolve(response)
output.normalize()

Here's what it looks like:

In [ ]:
wave.plot(label='original')
output.plot(label='convolved')
thinkplot.config(xlabel='time (s)', ylabel='amplitude', ylim=[-1.05, 1.05])

And here's what it sounds like:

In [ ]:
output.make_audio()

If you think this example is black magic, you are not alone.   But there is a good reason why this works, and I do my best to explain it in Chapter 9.  So stay tuned.

I'd like to thanks jcveliz and kleeb for making these recordings available from freesound.org.